## Setup

In [ ]:
import os

# importing scripting dependencies
import boa
from eth_account import Account

In [ ]:
from dotenv import load_dotenv

load_dotenv()
PRIVATE_KEY = os.environ.get("WEB3_TESTNET_PK")
# RPC_URL = "https://sepolia.optimism.io"
RPC_URL = "https://sepolia.base.org"
# RPC_URL = "https://mainnet.optimism.io"

## Prepare deployer and RPC

In [ ]:
# production mode (to deploy on ethereum)
deployer = Account.from_key(PRIVATE_KEY)
eth_env = boa.set_network_env(RPC_URL)
# boa.set_env(eth_env)

# this automatically sets the eoa as the deployer
boa.env.add_account(deployer)
print(f"Deploying with {deployer.address}")

# I. Deployment

### 1. ExampleMessenger

In [ ]:
# II. Then deploy the DepositLimitModule
contract_deployer = boa.load_partial("../contracts/ExampleMessenger.vy")
LZ_ENDPOINT_BASE_SEPOLIA = "0x6EDCE65403992e310A62460808c4b910D972f10f"
LZ_ENDPOINT_ID = 40245

contract = contract_deployer(
    LZ_ENDPOINT_BASE_SEPOLIA,  # endpoint on sepolia base
    500_000,  # default gas limit
)

print(f"Example Messenger deployed at {contract.address}")

## II. Post-deployment interactions 
## (web3py to simulate real interactions)

### 0. Prepare infra

In [ ]:
from web3 import Web3
import os
import logging
import subprocess

LZ_ENDPOINT_ID = 40245
LZ_CHAIN_ID = 84532


def get_vyper_abi():
    command = ["vyper", "../contracts/ExampleMessenger.vy", "-f", "abi_python"]
    try:
        result = subprocess.run(command, capture_output=True, text=True, check=True)
        return result.stdout
    except subprocess.CalledProcessError as e:
        return f"Error: {e.stderr}"


logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(message)s")
# RPC endpoints

CONTRACT_ADDRESS = contract.address

try:
    pk = os.environ["WEB3_TESTNET_PK"]
except KeyError:
    raise EnvironmentError("Please set WEB3_TESTNET_PK environment variable")
# execute vyper contracts/LZContract.vy -f abi_python in cli and get the output
ABI = get_vyper_abi()

w3 = Web3(Web3.HTTPProvider("https://sepolia.base.org"))
contract_w3 = w3.eth.contract(address=CONTRACT_ADDRESS, abi=ABI)
account = w3.eth.account.from_key(pk)

## 1. LZ Send & Receive

In [ ]:
# add self as peer

func = contract_w3.functions.set_peer(LZ_ENDPOINT_ID, CONTRACT_ADDRESS)
tx = func.build_transaction(
    {"from": account.address, "nonce": w3.eth.get_transaction_count(account.address)}
)
tx["gas"] = int(5 * w3.eth.estimate_gas(tx))
signed_tx = w3.eth.account.sign_transaction(tx, private_key=account.key)
tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)
logging.info(f"Tx: {tx_hash.hex()}")

In [ ]:
gas_limit = 500_000
msg = "170475436437825620930817601234267694881687829390282260281137596999800372275961"

fee = contract_w3.functions.quote_lz_fee(
    LZ_ENDPOINT_ID, CONTRACT_ADDRESS, msg.encode(), _gas_limit=gas_limit
).call()
logging.info(f"Fee: {fee}")

func = contract_w3.functions.send_message(
    _dst_eid=LZ_ENDPOINT_ID, _receiver=CONTRACT_ADDRESS, _message=msg, _gas_limit=5 * gas_limit
)
tx = func.build_transaction(
    {
        "from": account.address,
        "nonce": w3.eth.get_transaction_count(account.address),
        "value": 10 * fee,
    }
)
tx["gas"] = int(5 * w3.eth.estimate_gas(tx))
signed_tx = w3.eth.account.sign_transaction(tx, private_key=account.key)
tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)
logging.info(f"Tx: {tx_hash.hex()}")
# after we sent the message we can check it on testnet.layerzeroscan.com (use address or txid)
# shortly after, it should be pinged back (we send to ourselves, and event be fired)

## 2. LzRead

In [ ]:
# set self as read peer
LZ_READ_CHANNEL = 4294967294
func = contract_w3.functions.set_peer(LZ_READ_CHANNEL, CONTRACT_ADDRESS)
tx = func.build_transaction(
    {"from": account.address, "nonce": w3.eth.get_transaction_count(account.address)}
)
tx["gas"] = int(5 * w3.eth.estimate_gas(tx))
signed_tx = w3.eth.account.sign_transaction(tx, private_key=account.key)
tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)
logging.info(f"Set peer tx: {tx_hash.hex()}")

In [ ]:
# prepare reading calldata
from vyper.utils import method_id

method_str = "dummy_endpoint(uint256)"
num = 12345
calldata = method_id(method_str) + boa.util.abi.abi_encode("(uint256)", (num,))

# read the contract itself
res = contract_w3.functions.dummy_endpoint(num).call()

logging.info(f"Read: {res}")

# now quote lzread fee
fee = contract_w3.functions.quote_read_fee(
    LZ_ENDPOINT_ID, CONTRACT_ADDRESS, calldata, _gas_limit=gas_limit
).call()
logging.info(f"Fee: {fee}")


# now request read
func = contract_w3.functions.request_lzRead(
    LZ_ENDPOINT_ID, CONTRACT_ADDRESS, calldata, _gas_limit=gas_limit
).call()

tx = func.build_transaction(
    {
        "from": account.address,
        "nonce": w3.eth.get_transaction_count(account.address),
        "value": int(fee * 3),
    }
)
tx["gas"] = int(5 * w3.eth.estimate_gas(tx))
signed_tx = w3.eth.account.sign_transaction(tx, private_key=account.key)
tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)
logging.info(f"Tx: {tx_hash.hex()}")